# News content extraction

In [ ]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
import time
import os
from random import uniform, random, choice, shuffle
import pandas as pd
from tqdm import tqdm
import pickle

In [ ]:
file_name = 'wsj_tesla_links.txt' #replace with the file name of the links you want to extract


dir_path = f"../data/Links for Finetuning Language Models/"
file_path = os.path.join(dir_path, file_name)
#import a list of links here
with open(file_path, 'r') as f:
    links = f.readlines()
print(len(links))

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [ ]:
def create_webdriver():
    return webdriver.chrome.webdriver.WebDriver()

In [ ]:
%load_ext dotenv
%dotenv

email = %env EMAIL
PW = %env PASSWORD 

In [ ]:
browser = create_webdriver()
barrons_link = "https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.barrons.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-barrons-223-2&nonce=5be85805-8aae-4e29-879d-6443fcc540a7&state=7jxTAzU1TvYYT8DZ.YXZXI-6hSh7SUwIahTuw8TWcpS5djZgQE4b5PIUveh8&resource=https%253A%252F%252Fwww.barrons.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
wsj_link = "https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.wsj.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-wsj-223-2&nonce=52c47041-ffac-4ebd-a213-e777ff5b684e&state=Br3BYl9iSECXktJn.c8ksuGzlG5CBOw5mEnxqxAx_vb3PwwuwocDA8Tqt_CU&resource=https%253A%252F%252Fwww.wsj.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
marketwatch_link = "https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20updated_at%20createTimestamp&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.marketwatch.com%2Fauth%2Fsso%2Flogin&nonce=1c401a0c-e50d-4ff9-b9e5-9ee66d23b69b&ui_locales=en-us-x-mw-223-2&ns=prod%2Faccounts-mw&state=Ij_BjehgojKihO2D.fol-KzuuFbZNtMy6SZz5wCTpnK7bXYKUflGlSLfPKYg&resource=https%253A%252F%252Fwww.marketwatch.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
link = wsj_link #choose provider here
browser.get(link)

In [ ]:
email_field = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.ID, "username")))
email_field.send_keys(email)

In [ ]:
login_button = browser.find_element(By.XPATH, '//*[@id="password-login"]/div/form/div/div[5]/div[1]/button')
browser.execute_script("arguments[0].click();", login_button);

In [ ]:

cnt = 0
data = {'link': [], 'topic': [], 'time': [], 'header': [], 'content': []}
with tqdm(total = len(links)) as pbar:
    for ind, i in enumerate(links):
        pbar.update(1)
        browser.get(i)
        print('ind:',ind)
        print(i)
        tmp_id = i.split('/')[-1].split('?')[0]
#         print(tmp_id)
        try:
            time.sleep(1.5)
            tmp_date = browser.find_element(By.XPATH,f'//*[@id="{tmp_id}"]/div[1]/div[4]/div/p').text
            header = browser.find_elements(By.TAG_NAME, 'h1')[0].text
            contents = [i.text for i in browser.find_elements(By.TAG_NAME, 'P')]
            category = browser.find_elements(By.TAG_NAME, 'NAV')[0].text

        except:
            try: 
                time.sleep(1)
                tmp_date = browser.find_element(By.XPATH,f'//*[@id="{tmp_id}"]/div[1]/div[4]/div/p').text
                header = browser.find_elements(By.TAG_NAME, 'h1')[0].text
                contents = [i.text for i in browser.find_elements(By.TAG_NAME, 'P')]
                category = browser.find_elements(By.TAG_NAME, 'NAV')[0].text
            except:
                print('elem.text: broken')
                cnt += 1
                continue
        print('header:', header)
        data['link'].append(i)
        data['topic'].append(category)
        data['time'].append(tmp_date)
        data['header'].append(header)
        data['content'].append(contents)